In [ ]:
# from google.colab import drive
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
drive.mount('/content/drive')
# Create a directory in Drive for Hugging Face cache
HF_CACHE_DIR = "/content/drive/MyDrive/hf_cache"
os.makedirs(HF_CACHE_DIR, exist_ok=True)

# Set environment variable so Hugging Face uses this cache
os.environ["TRANSFORMERS_CACHE"] = HF_CACHE_DIR

In [ ]:
! cd drive/MyDrive/dataliteracy25

In [ ]:
from scripts import xml_reader, xml_scrapper

# xml_scrapper.get_all_sittings(out_folder="data/xml")

# os.makedirs("data/csv/10", exist_ok=True)

for root, dirs, files in os.walk("data/xml/10"): 
    for f in files: 
        print(f)
        contributions = xml_reader.parse_contributions(f"{root}/{f}")
        
        df = pd.DataFrame(contributions)
        
        df = df[~(df["PP"] == "NULL")]
        
        df.to_csv(f"data/csv/10/{f}".replace(".xml", ".csv"))

In [ ]:
# Load model directly
from transformers import AutoModel
model = AutoModel.from_pretrained("jinaai/jina-embeddings-v4", trust_remote_code=True, torch_dtype="auto").to("cuda")

def embedd_speech(df, key, out_path):
    embeddings = model.encode_text(texts=[c[key] for _, c in df.iterrows()], task="text-matching", prompt_name="query", return_numpy=True)
    np.save(out_path, embeddings)
    return embeddings

In [ ]:
import seaborn as sns
import matplotlib.pylab as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import json 

color_map = json.load(open("data/map_groups_color.json"))
os.makedirs("data/plots/10", exist_ok=True)

def plot_and_cluster(title, embeddings, texts, filename, k_clusters=5):
  pca = PCA(n_components=2)
  reduced = pca.fit_transform(embeddings)

  # 2. KMeans clustering
  kmeans = KMeans(n_clusters=k_clusters, random_state=42)
  labels = kmeans.fit_predict(reduced)

  # 3. Plot
  plt.figure(figsize=(10, 8))

  scatter = plt.scatter(reduced[:,0], reduced[:,1], c=labels, cmap='tab10', s=150)
  plt.title(title)

  # 4. Add text labels
  for i, txt in enumerate(texts):
      plt.text(reduced[i,0]+0.02, reduced[i,1]+0.02, txt, fontsize=12, color=color_map[txt])

  plt.legend(*scatter.legend_elements(), title="Cluster")
  plt.savefig(f"data/plots/10/{filename}")

In [ ]:
for root, dirs, files in os.walk("data/csv/10"): 
    for f in files: 
        df = pd.read_csv(f"{root}/{f}")
        embedding = embedd_speech(df, "speech", f"data/embeddings/10/{f.replace(".csv", ".npy")}")
        plot_and_cluster(f, embedding, df["PP"], f.replace(".csv", ".svg"), k_clusters=len(df)/10)

NameError: name 'embedd_speech' is not defined